In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/CS229Buddies/project')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import collections
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, f1_score, plot_precision_recall_curve
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import operator
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
!pip install gseapy
import gseapy

     |████████████████████████████████| 532kB 5.3MB/s 
     |████████████████████████████████| 235kB 49.8MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 153kB 45.1MB/s 
     |████████████████████████████████| 5.6MB 44.4MB/s 
     |████████████████████████████████| 256kB 50.9MB/s 
  Created wheel for bioservices: filename=bioservices-1.7.11-cp37-none-any.whl size=236089 sha256=f8d650e09eec3de4d1fc79f4084b8ed732bfbd659af633e6677a1f2db64648c8
  Stored in directory: /root/.cache/pip/wheels/60/6c/61/1e0bf60bbf1c01bebc0b36f25aa94a4028474203414ed4088d
  Created wheel for easydev: filename=easydev-0.11.1-cp37-none-any.whl size=64252 sha256=b03b9f0df2b75e7b1cdd3748f8c1076978c39123af9e0646e88bd20239753fd6
  Stored in directory: /root/.cache/pip/wheels/d8/1d/3b/4e589dff14dc2dce911de5c951c371c99e0da29f9f733dfbd2
  Created wheel for suds-jurko: filename=suds_jurko-0.6-cp37-none-any.whl size=454361 sha256=ef1c62cbf52562ddcae0ab46c6db6a94ebbb8a54eb0

In [ ]:
df = pd.read_csv('data/GSE113957_fpkm.txt', sep = '\t')
df = df.T
transcript_ids = list(df.loc['Transcript ID'])

df = df.drop(['Transcript ID', 'chr', 'start', 'end', 'strand', 'Length', 'Copies', 'Annotation/Divergence'], axis=0)
df['pids'] = [column.lower().split('_')[0] for column in df.index]
hgps = set(['139', '141', '197', '198', '199', '200', '201', '202', '203', '204'])
control_df = df[~df.pids.isin(hgps)]
hgps_df = df[df.pids.isin(hgps)]
control_df = control_df.drop(['pids'], axis = 1)
hgps_df = hgps_df.drop(['pids'], axis = 1)

control_df.columns = transcript_ids
hgps_df.columns = transcript_ids

total_df = control_df.append(hgps_df)

In [ ]:
annot=pd.read_csv("data/uniprot.tab", sep='\t')
annot

,yourlist:M202105245C475328CEF75220C360D524E9D456CE20BD46G,isomap:M202105245C475328CEF75220C360D524E9D456CE20BD46G,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,NM_173803,NM_173803 -> Q2QL34-2,Q2QL34,MP17L_HUMAN,reviewed,Mpv17-like protein (M-LP homolog) (M-LPH),MPV17L,Homo sapiens (Human),196
1,NM_014423,NM_014423 -> Q9UHB7-1,Q9UHB7,AFF4_HUMAN,reviewed,AF4/FMR2 family member 4 (ALL1-fused gene from...,AFF4 AF5Q31 MCEF HSPC092,Homo sapiens (Human),1163
2,NM_001103167,NaN,P0C6A0,ZGLP1_HUMAN,reviewed,GATA-type zinc finger protein 1 (GATA-like pro...,ZGLP1 GLP1,Homo sapiens (Human),271
3,NM_018397,NaN,Q8NE62,CHDH_HUMAN,reviewed,"Choline dehydrogenase, mitochondrial (CDH) (CH...",CHDH,Homo sapiens (Human),594
4,NM_001037671,NM_001037671 -> Q32Q52-1,Q32Q52,CL074_HUMAN,reviewed,Uncharacterized protein C12orf74,C12orf74,Homo sapiens (Human),190
...,...,...,...,...,...,...,...,...,...
24278,NM_021923,NaN,Q8N441,FGRL1_HUMAN,reviewed,Fibroblast growth factor receptor-like 1 (FGF ...,FGFRL1 FGFR5 FHFR UNQ480/PRO943,Homo sapiens (Human),504
24279,NM_017417,NaN,Q9NY28,GALT8_HUMAN,reviewed,Probable polypeptide N-acetylgalactosaminyltra...,GALNT8,Homo sapiens (Human),637
24280,NM_004354,NM_004354 -> Q16589-1,Q16589,CCNG2_HUMAN,reviewed,Cyclin-G2,CCNG2,Homo sapiens (Human),344
24281,NM_004354,NaN,A0A024RDC7,A0A024RDC7_HUMAN,unreviewed,"Cyclin G2, isoform CRA_a",CCNG2 hCG_22999,Homo sapiens (Human),344


In [ ]:
annot=pd.read_csv("data/uniprot.tab", sep='\t')
name_d=pd.Series(annot["Gene names"].values,index=annot["yourlist:M202105245C475328CEF75220C360D524E9D456CE20BD46G"]).to_dict()

In [ ]:
n=dict()
for k,v in name_d.items():
  if str(v)!="nan" and str(k)!="nan":
    n[k]=v.split(" ")[0]

In [ ]:
n

{'NM_173803': 'MPV17L',
 'NM_014423': 'AFF4',
 'NM_001103167': 'ZGLP1',
 'NM_018397': 'CHDH',
 'NM_001037671': 'C12orf74',
 'NM_133458': 'ZFP90',
 'NM_001080424': 'KDM6B',
 'NM_001271816': 'DCLRE1A',
 'NM_001195545': 'LRRC3C',
 'NM_014879': 'P2RY14',
 'NM_006918': 'SC5DL',
 'NM_001145318': 'DSN1',
 'NM_001135914': 'KCP',
 'NM_170679': 'SKP1',
 'NM_001318781': 'ADAMTS9',
 'NM_004698': 'PRPF3',
 'NM_017554': 'PARP14',
 'NM_002144': 'HOXB1',
 'NM_001282763': 'CDV3',
 'NM_005663': 'NELFA',
 'NM_002559': 'P2RX3',
 'NM_015902': 'UBR5',
 'NM_001042529': 'COASY',
 'NM_001039569': 'AP1S3',
 'NM_001025205': 'AP2M1',
 'NM_001004351': 'SPDYE3',
 'NM_020940': 'FAM160B1',
 'NM_001146685': 'TMEM88B',
 'NM_002886': 'RAP2B',
 'NM_031415': 'GSDMC',
 'NM_018147': 'FAIM',
 'NM_001079528': 'ABCC6',
 'NM_001014442': 'PCOTH',
 'NM_001199723': 'CRABP2',
 'NM_052889': 'CARD16',
 'NM_001320878': 'SULT1C3',
 'NM_032860': 'LTV1',
 'NM_001330343': 'AID',
 'NM_001308203': 'SGIP1',
 'NM_001144070': 'ABCC3',
 'NM_001

In [ ]:
total_df=total_df[[x for x in total_df.columns if x  in n.keys()]]
total_df.columns=[n[x] for x in total_df.columns if x  in n.keys()]

In [ ]:
total_df=total_df.T
total_df

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,69_87_Female_caucasian,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown
MPV17L,0.133,0.213,0.054,0.066,0.156,0.196,0.193,0.149,0.059,0.025,0.15,0.069,0.022,0.079,0.098,0,0.314,0.102,0.083,0.104,0.115,0.14,0.122,0.179,0.203,0.107,0.039,0.299,0.047,0.094,0.162,0.074,0.038,0.209,0.023,0.246,0.139,0.096,0.148,0.173,...,0.058,0.074,0.09,0.16,0.117,0.134,0.108,0.062,0.037,0.289,0.093,0.158,0.055,0,0.085,0.103,0.19,0.257,0.358,0.389,0.056,0.062,0.218,0.072,0.151,0.214,0.041,0.204,0.177,0,0.277,0.068,0.164,0.204,0.107,0.147,0.18,0.068,0.08,0.067
AFF4,16.98,16.772,16.206,15.345,16.873,19.268,14.548,21.798,20.975,16.427,15.902,14.852,18.592,16.316,17.847,16.728,20.295,17.772,16.225,20.292,20.173,18.427,21.683,20.296,19.14,21.458,17.768,23.935,19.454,20.497,18.454,24.81,21.377,17.884,15.925,15.074,17.602,14.818,16.508,16.472,...,20.426,10.808,19.808,25.843,15.358,26.818,29.769,22.594,26.124,26.493,19.009,15.875,19.405,14.743,18.111,17.369,14.234,15.411,13.617,12.931,14.306,12.998,16.116,9.253,15.375,16.103,15.899,16.79,9.796,14.04,18.963,15.965,19.677,18.016,18.419,17.866,18.088,18.901,21.639,17.284
ZGLP1,0.387,0.168,0.41,0.148,0.252,0.387,0.298,0.252,0.365,0.626,0.846,0.365,0.177,0.238,0.638,0.676,0.503,0.461,0.969,0.412,0.801,0.182,0.344,0.383,0.409,0.243,0.508,0.26,0.349,0.4,0.488,0.391,0.255,0.568,0.504,0.361,0.433,0.415,0.538,0.445,...,0.197,0.334,0.475,0.181,0.264,0.302,0.122,0.327,0.042,0.286,0.493,0.268,0.596,0.292,0.289,0.444,0.583,0.554,0.686,0.733,0.377,0.538,1.014,0.324,0.409,0.323,0.486,0.385,0.452,0.55,0.333,0.402,0.386,0.51,0.418,0.362,0.435,0.48,0.291,0.331
CHDH,0.115,0.077,0.284,0.304,0.094,0.118,0.276,0.09,0.071,0.365,0.166,0.153,0.217,0.159,0.044,0.095,0.142,0.169,0.067,0.378,0.046,0.437,0.275,0.289,0.123,1.621,1.76,0.974,0.974,1.039,2.053,0.478,0.306,0.354,0.496,0.166,0.021,0.135,0.153,0.095,...,3.373,2.925,3.587,2.123,0.141,2.889,0.174,3.964,2.22,0.044,0.169,0.064,0.067,0.109,0.18,0.037,0.049,0.155,0.033,0.065,0.458,0.075,0.073,0.065,0.109,0.086,0.087,0.014,0.147,0.214,0,0.398,0.132,0.202,0.151,0.056,0.054,0.05,0.058,0.04
C12orf74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...

In [ ]:
  l=gseapy.ssgsea(data=total_df.iloc[:,0], gene_sets='DisGeNET')

2021-05-30 06:50:19,924 Warning: dropping duplicated gene names, only keep the first values


KeyboardInterrupt: ignored

In [ ]:
patient_feature[col]=list(l.res2d["sample1"])

In [ ]:
print(l.res2d.index)

In [ ]:
l=list(total_df.sort_values("101_19yr_Female_Caucasian", axis=0, ascending=False).index)

In [ ]:
patient_feature=dict()
for col in list(total_df.columns):
  l=gseapy.ssgsea(data=total_df[col], gene_sets='Aging_Perturbations_from_GEO_down')
  patient_feature[col]=list(l.res2d["sample1"])

2021-05-30 06:27:38,737 Warning: dropping duplicated gene names, only keep the first values
2021-05-30 06:28:52,842 Warning: dropping duplicated gene names, only keep the first values
2021-05-30 06:29:47,570 Warning: dropping duplicated gene names, only keep the first values
2021-05-30 06:30:42,392 Warning: dropping duplicated gene names, only keep the first values


KeyboardInterrupt: ignored

In [ ]:
for col in list(total_df.columns)[141:]:
  l=gseapy.ssgsea(data=total_df[col], gene_sets='')
  patient_feature[col]=list(l.res2d["sample1"])






AttributeError: ignored

In [ ]:
list(total_df.columns)[141]

'203_8yr10mos_M_Unknown'

In [ ]:
pd.DataFrame.from_dict(patient_feature)

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,69_87_Female_caucasian,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown
0,0.350890,0.347854,0.323341,0.319741,0.348308,0.353414,0.350498,0.360435,0.360693,0.334585,0.331541,0.350204,0.354583,0.326394,0.327429,0.344968,0.331023,0.345603,0.340241,0.338566,0.341722,0.316734,0.353803,0.345665,0.354654,0.347367,0.344354,0.352872,0.347037,0.352841,0.338798,0.357623,0.358612,0.335348,0.332923,0.340830,0.338460,0.350980,0.324259,0.329552,...,0.334397,0.329551,0.330741,0.372936,0.319973,0.357746,0.393503,0.347189,0.367100,0.369298,0.335381,0.348068,0.343269,0.329249,0.337968,0.354243,0.352401,0.335017,0.328180,0.350953,0.342896,0.333524,0.348205,0.323334,0.321925,0.333707,0.344194,0.355421,0.327752,0.353976,0.339281,0.333005,0.351428,0.353690,0.335838,0.333286,0.345568,0.338217,0.351784,0.331427
1,0.055188,0.045778,0.031528,0.066636,0.055484,0.070795,0.040182,0.052664,0.060449,0.055535,0.072680,0.034272,0.037635,0.064548,0.053665,0.075708,0.057642,0.047385,0.055292,0.048208,0.060454,0.085034,0.030776,0.077477,0.073350,0.057372,0.048812,0.094054,0.062942,0.050739,0.068135,0.086050,0.077861,0.046128,0.049494,0.045037,0.063819,0.049119,0.072949,0.071956,...,0.057631,0.055631,0.059293,0.067867,0.060351,0.049436,0.073468,0.054789,0.058979,0.088935,0.028985,0.049073,0.044907,0.052374,0.056514,0.049705,0.027058,0.065638,0.071953,0.039755,0.046988,0.066496,0.051579,0.059843,0.072234,0.037804,0.062369,0.053469,0.051053,0.040294,0.068393,0.024957,0.041997,0.027829,0.067077,0.034187,0.039675,0.046186,0.063958,0.051870
2,0.386474,0.375981,0.371075,0.369548,0.381344,0.390262,0.382853,0.380089,0.384769,0.376531,0.376737,0.372272,0.388178,0.384520,0.383952,0.379058,0.376365,0.374223,0.389405,0.378269,0.368857,0.378652,0.367864,0.365541,0.382766,0.374305,0.379177,0.365543,0.376188,0.378731,0.374810,0.381255,0.368398,0.380297,0.394586,0.393355,0.387522,0.374710,0.382578,0.385260,...,0.390229,0.392053,0.381103,0.393404,0.386679,0.386918,0.397023,0.390504,0.386948,0.383626,0.394927,0.388429,0.389651,0.384742,0.403174,0.379645,0.395807,0.375853,0.375637,0.387032,0.400884,0.380316,0.394208,0.379312,0.387790,0.380689,0.395699,0.375380,0.381486,0.399839,0.396901,0.389534,0.403314,0.398437,0.387310,0.390094,0.395697,0.393274,0.394334,0.3

In [ ]:
gsea_output=pd.DataFrame.from_dict(patient_feature)
gsea_output.index=l.res2d.index
gsea_output.to_csv("data/GO_matrix")

In [ ]:
gsea_output

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,69_87_Female_caucasian,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown
Term|NES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'de novo' posttranslational protein folding (GO:0051084),0.350890,0.347854,0.323341,0.319741,0.348308,0.353414,0.350498,0.360435,0.360693,0.334585,0.331541,0.350204,0.354583,0.326394,0.327429,0.344968,0.331023,0.345603,0.340241,0.338566,0.341722,0.316734,0.353803,0.345665,0.354654,0.347367,0.344354,0.352872,0.347037,0.352841,0.338798,0.357623,0.358612,0.335348,0.332923,0.340830,0.338460,0.350980,0.324259,0.329552,...,0.334397,0.329551,0.330741,0.372936,0.319973,0.357746,0.393503,0.347189,0.367100,0.369298,0.335381,0.348068,0.343269,0.329249,0.337968,0.354243,0.352401,0.335017,0.328180,0.350953,0.342896,0.333524,0.348205,0.323334,0.321925,0.333707,0.344194,0.355421,0.327752,0.353976,0.339281,0.333005,0.351428,0.353690,0.335838,0.333286,0.345568,0.338217,0.351784,0.331427
3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427),0.055188,0.045778,0.031528,0.066636,0.055484,0.070795,0.040182,0.052664,0.060449,0.055535,0.072680,0.034272,0.037635,0.064548,0.053665,0.075708,0.057642,0.047385,0.055292,0.048208,0.060454,0.085034,0.030776,0.077477,0.073350,0.057372,0.048812,0.094054,0.062942,0.050739,0.068135,0.086050,0.077861,0.046128,0.049494,0.045037,0.063819,0.049119,0.072949,0.071956,...,0.057631,0.055631,0.059293,0.067867,0.060351,0.049436,0.073468,0.054789,0.058979,0.088935,0.028985,0.049073,0.044907,0.052374,0.056514,0.049705,0.027058,0.065638,0.071953,0.039755,0.046988,0.066496,0.051579,0.059843,0.072234,0.037804,0.062369,0.053469,0.051053,0.040294,0.068393,0.024957,0.041997,0.027829,0.067077,0.034187,0.039675,0.046186,0.063958,0.051870
7-methylguanosine RNA capping (GO:0009452),0.386474,0.375981,0.371075,0.369548,0.381344,0.390262,0.382853,0.380089,0.384769,0.376531,0.376737,0.372272,0.388178,0.384520,0.383952,0.379058,0.376365,0.374223,0.389405,0.378269,0.368857,0.378652,0.367864,0.365541,0.382766,0.374305,0.379177,0.365543,0.376188,0.378731,0.374810,0.381255,0.368398,0.380297,0.394586,0.393355,0.387522,0.374710,0.382578,0.385260,...,0.390229,0.392053,0.381103,0.393404,0.386679,0.386918,0.397023,0.390504,0.386948,0.383626,0.394927,0.

In [ ]:
 gsea_output=pd.DataFrame.from_dict(patient_feature)
gsea_output.index=l.res2d.index
gsea_output.to_csv("data/KEGG_matrix")

In [ ]:
gseapy.ssgsea(data=total_df["101_19yr_Female_Caucasian"], gene_sets='KEGG_2016', outdir='gseapy')
pd.read_csv("gseapy/gseapy.samples.normalized.es.txt", skiprows=3, sep='\t', header=None)

2021-05-24 16:21:18,763 Warning: dropping duplicated gene names, only keep the first values


,0,1
0,ABC transporters Homo sapiens hsa02010,-0.12388202986963145
1,AGE-RAGE signaling pathway in diabetic complic...,0.25365177227014957
2,AMPK signaling pathway Homo sapiens hsa04152,0.1893446631313747
3,Acute myeloid leukemia Homo sapiens hsa05221,0.26509192685704885
4,Adherens junction Homo sapiens hsa04520,0.3032962647756284
...,...,...
1088,cAMP signaling pathway Homo sapiens hsa04024,0.030048616927829192
1089,cGMP-PKG signaling pathway Homo sapiens hsa04022,0.10161078673831658
1090,mRNA surveillance pathway Homo sapiens hsa03015,0.33073507447512346
1091,mTOR signaling pathway Homo sapiens hsa04150,0.2338191651425188


In [ ]:
l=gseapy.ssgsea(data=total_df["102_19yr_Male_Caucasian"], gene_sets='GO_Biological_Process_2018', outdir='gseapy')
# l=pd.read_csv("gseapy/gseapy.samples.normalized.es.txt", skiprows=3, sep='\t', header=None)[1]

2021-05-24 19:05:40,899 Warning: dropping duplicated gene names, only keep the first values


In [ ]:
l.res2d

,sample1
Term|NES,
'de novo' posttranslational protein folding (GO:0051084),0.347854
3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427),0.045778
7-methylguanosine RNA capping (GO:0009452),0.375981
7-methylguanosine mRNA capping (GO:0006370),0.375981
ATP generation from ADP (GO:0006757),0.129819
...,...
"wound healing, spreading of cells (GO:0044319)",0.239103
zinc II ion transmembrane transport (GO:0071577),0.109173
zinc II ion transport (GO:0006829),0.141692


In [ ]:
l=gseapy.ssgsea(data=total_df["101_19yr_Female_Caucasian"], gene_sets='KEGG_2019_Human', outdir='gseapy')

2021-05-24 17:20:22,541 Warning: dropping duplicated gene names, only keep the first values


In [ ]:
l.res2d

,sample1
Term|NES,
ABC transporters,-0.129857
AGE-RAGE signaling pathway in diabetic complications,0.258711
AMPK signaling pathway,0.193693
Acute myeloid leukemia,0.201347
Adherens junction,0.309602
...,...
cAMP signaling pathway,0.008684
cGMP-PKG signaling pathway,0.084198
mRNA surveillance pathway,0.331145


In [ ]:
l=pd.read_csv("gseapy/gseapy.samples.normalized.es.txt", skiprows=1, na_values=["sample1", "# normalize enrichment scores by using the entire data set", ""], sep='\t', header=1).iloc[:,1]

In [ ]:
import gseapy

Creating directory /root/.config/bioservices 


In [ ]:
import math

In [ ]:
cdata=pd.read_csv("data/Cerebrum.csv")
age=[math.floor(float(x)) for x in list(cdata.columns)[3:]]
age

[1,
 1,
 1,
 1,
 1,
 6,
 6,
 6,
 6,
 6,
 16,
 16,
 16,
 16,
 16,
 24,
 24,
 24,
 24,
 24,
 1,
 1,
 1,
 1,
 1,
 6,
 6,
 6,
 6,
 6,
 16,
 16,
 16,
 16,
 16,
 24,
 24,
 24,
 24,
 24]

In [ ]:
cdata

,Unnamed: 0,Unnamed: 1,Age (mos.),1,1.1,1.2,1.3,1.4,6,6.1,6.2,6.3,6.4,16,16.1,16.2,16.3,16.4,24,24.1,24.2,24.3,24.4,1.5,1.6,1.7,1.8,1.9,6.5,6.6,6.7,6.8,6.9,16.5,16.6,16.7,16.8,16.9,24.5,24.6,24.7,24.8,24.9
0,Mouse Unigene,Gene Name,Gene Ontology,Expressionb,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression,Expression
1,Mm.10,Srm,molecular function|S-adenosylmethionine-depend...,-0.4753761,-0.2556392,-0.5178132,-0.4449675,-0.4407368,-0.6612224,-0.0954796,-0.5295781,-0.3421829,-0.7270489,-0.5572581,-0.4661075,-0.4590636,-0.6091528,-0.6429578,-0.7888984,-0.4386265,-0.6893008,-0.3895278,-0.8610118,-0.5469843,-0.9176563,-0.6220961,-0.7368635,-0.5224012,-0.6833717,-0.439732,-0.3725527,-0.6953695,-0.5859947,-0.4451592,-0.4216521,-0.5916838,-0.503354,-0.74654,-0.5609004,-0.477177,-0.4825825,-0.303152,-0.409863
2,Mm.100065,1300012G16Rik,cellular component|extracellular space|TAS|GO:...,0.83780835,0.83346743,0.90748021,0.86945359,1.02229927,1.10546312,1.36475431,0.77315234,0.90230051,0.75075735,0.68740851,1.27847646,1.3792745,0.42356348,0.67906685,1.39887175,1.00013633,0.90489427,0.83995912,1.32705851,1.02501671,0.84384193,0.85960064,1.05262554,1.08841716,0.91518722,0.99454858,0.6711797,0.82980748,0.92698416,0.6834216,1.13565208,1.28383709,1.24353925,0.89216597,1.11337896,0.78136164,1.33795645,1.44129678,0.8433223
3,Mm.100068,Amot,biological process|cell migration during gastr...,0.07717648,0.01332556,0.07726495,-0.2916745,-0.018306,-0.1875757,-0.3114304,0.00451575,-0.0986918,0.01619811,0.06087283,-0.2069515,-0.0097121,0.06685635,0.14117691,-0.3048347,0.07622446,0.01406061,-0.2010302,-0.100795,0.02183513,0.16870956,-0.3108919,-0.0259094,0.05753805,-0.2250655,-0.1054719,-0.126283,-0.0024876,-0.1419874,-0.2109417,0.00226821,-0.3061982,-0.4278881,0.17369803,0.06517981,-0.0286165,-0.1452634,-0.3802045,0.01515834
4,Mm.100112,Brwd3,biological process|biological process unknown|...,0.59576972,0.6172773,0.46812061,0.57323495,0.53796095,0.44606193,0.49805686,0.42196721,0.56931397,0.62742933,0.45146704,0.42010234,0.34298737,0.64037749,0.56367722,0.13947009,0.47545406,0.40790473,0.47832834,0.28519382,0.55479579,0.44285469,0.26208679,0.24340415,0.36226091,0.59441195,0.481242,0.55948405,0.56044397,0.36596194,0.35153893,0.72096356,0.50287585,0.58610982,0.41254781,0.35858276,0.59805624,0.51049015,0.18679306,0.4423562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8928,Mm.998,Psen1,cellular component|Golgi apparatus|IEA|GO:0005...,1.185673705,1.030635855,1.31659208,0.665270345,1.1911239,1.15475701,1.136131225,1.14273945,1.312616295,1.18531147,1.20995463,0.925364205,1.09513931,1.368485525,1.206472395,0.65619336,1.2118898,1.317626665,1.222110735,1.10461606,1.25414927,1.210896525,0.509877015,0.991039155,1.180318615,0.699967275,1.07146211,1.252613205,0.94247728,1.20854823,1.19412915,1.41567245,1.0432137,0.955767675,1.212382275,1.133628495,1.195023355,1.03187094,0.95596497,1.27321269
8929,Mm.99850,E130014J05Rik,NaN,-0.3103466,-0.5534943,-0.5174676,-1.6197507,-0.5008236,-1.6813802,-1.5599422,-0.0887347,-0.6007312,-0.7321595,-0.3991983,-1.5764923,-0.9226927,-0.670649,-0.5346593,-2.1688847,-0.5755651,-0.5590077,-0.6039836,-1.5769135,-0.6209932,-0.4096192,-1.5108105,-0.1152526,-0.6892428,-1.4022443,-0.9203527,-0.6373039,-0.3803616,-0.4320823,-0.1547424,-0.6614554,-1.92966,-1.69862,-0.2379645,-0.3882037,-0.4803873,-1.6769751,-1.9873955,-0.3087691
8930,Mm.999,Mpv17,cellular component|integral to membrane|IEA|GO...,-0.094848057,0.09267808,-0.034679597,0.15825672,0.072524103,0.

In [ ]:
cdata=cdata.iloc[1:,:]

In [ ]:
cdata.iloc[0,2].split("//")

['molecular function|S-adenosylmethionine-dependent methyltransferase activity|IEA|GO:0008757|MGI|na',
 'molecular function|catalytic activity|IEA|GO:0003824|MGI|na',
 'biological process|spermidine biosynthesis|IEA|GO:0008295|MGI|na',
 'molecular function|spermidine synthase activity|IEA|GO:0004766|MGI|na',
 'molecular function|transferase activity|IEA|GO:0016740|MGI|na']

In [ ]:
gseapy.get_library_name()

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'COVID-19_Related_Gene_Sets',
 'Cancer_Cell_Line_Encyclopedia',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
 'Descartes_Cell_Types_and_Tissue_2021',
 'DisGeNET',
 'Disease_Perturbations_from_GEO_down',
 'Disease_Perturbations_from_GEO_up',
 'Disease_Signatures_from_GEO_down_2014',
 'Disease_Signatures_from_GEO_up_2